In [44]:
import clingo
import clingo.ast

In [45]:
program = """

domA(0).
domA(1).

domB(1).
domB(2).

join(A,B) :- domA(A), domB(B).

"""

In [46]:
pos = clingo.ast.Position('<string>', 1, 1)
loc = clingo.ast.Location(pos, pos)

domA0 = clingo.ast.Literal(loc, clingo.ast.Sign.NoSign, clingo.ast.SymbolicAtom(clingo.ast.SymbolicTerm(loc, clingo.Function('domA', [clingo.Number(0)]))))
domA1 = clingo.ast.Literal(loc, clingo.ast.Sign.NoSign, clingo.ast.SymbolicAtom(clingo.ast.SymbolicTerm(loc, clingo.Function('domA', [clingo.Number(1)]))))
domB1 = clingo.ast.Literal(loc, clingo.ast.Sign.NoSign, clingo.ast.SymbolicAtom(clingo.ast.SymbolicTerm(loc, clingo.Function('domB', [clingo.Number(1)]))))
domB2 = clingo.ast.Literal(loc, clingo.ast.Sign.NoSign, clingo.ast.SymbolicAtom(clingo.ast.SymbolicTerm(loc, clingo.Function('domB', [clingo.Number(2)]))))

domAA = clingo.ast.Literal(loc, clingo.ast.Sign.NoSign,clingo.ast.SymbolicAtom(clingo.ast.Function(loc, 'domA', [clingo.ast.Variable(loc, 'A')], False)))
domBB = clingo.ast.Literal(loc, clingo.ast.Sign.NoSign,clingo.ast.SymbolicAtom(clingo.ast.Function(loc, 'domB', [clingo.ast.Variable(loc, 'B')], False)))

joinAB = clingo.ast.Literal(loc, clingo.ast.Sign.NoSign, clingo.ast.SymbolicAtom(clingo.ast.Function(loc, 'join', [clingo.ast.Variable(loc, 'A'), clingo.ast.Variable(loc, 'B')], False)))

nodes = [
    clingo.ast.Rule(loc, head=domA0, body=()),
    clingo.ast.Rule(loc, head=domA1, body=()),
    clingo.ast.Rule(loc, head=domB1, body=()),
    clingo.ast.Rule(loc, head=domB2, body=()),
    clingo.ast.Rule(loc, head=joinAB, body=[domAA, domBB])
]
print('\n'.join(map(str, nodes)))

domA(0).
domA(1).
domB(1).
domB(2).
join(A,B) :- domA(A); domB(B).


In [47]:
ctl = clingo.Control()
ctl.configuration.solve.models = 0

In [48]:
with clingo.ast.ProgramBuilder(ctl) as bld:
    for node in nodes:
        bld.add(node)

In [49]:
ctl.ground([('base', [])])
ctl.solve(on_model=print)

domB(1) domB(2) domA(0) domA(1) join(0,1) join(1,1) join(0,2) join(1,2)


SolveResult(5)

In [50]:
def report(node):
    print(node)
    if not isinstance(node, clingo.ast.AST):
        print("Type:", type(node).__name__)
        return

    print("Type:", node.ast_type)
    print("Keys:", node.keys())

    if 'head' in node.keys():
        print("Head:", end=' ')
        report(node.head)
    if 'body' in node.keys():
        print("Body:", end=' ')
        report(node.body)
    if 'atom' in node.keys():
        print("Atom: ", end='')
        report(node.atom)
    if 'symbol' in node.keys():
        print("Symbol: ", end='')
        report(node.symbol)
    if 'sign' in node.keys():
        print("Sign:", node.sign, type(node.sign).__name__)
    if 'external' in node.keys():
        print("External:", node.external, type(node.external).__name__)
    if 'name' in node.keys():
        print("Name:", node.name, type(node.name).__name__)
    if 'arguments' in node.keys():
        print("Arguments:")
        for argument in node.arguments:
            report(argument)


In [51]:
clingo.ast.parse_string("join(A,B) :- dom(A), dom(B).", report)

#program base.
Type: ASTType.Program
Keys: ['location', 'name', 'parameters']
Name: base str
join(A,B) :- dom(A); dom(B).
Type: ASTType.Rule
Keys: ['location', 'head', 'body']
Head: join(A,B)
Type: ASTType.Literal
Keys: ['location', 'sign', 'atom']
Atom: join(A,B)
Type: ASTType.SymbolicAtom
Keys: ['symbol']
Symbol: join(A,B)
Type: ASTType.Function
Keys: ['location', 'name', 'arguments', 'external']
External: 0 int
Name: join str
Arguments:
A
Type: ASTType.Variable
Keys: ['location', 'name']
Name: A str
B
Type: ASTType.Variable
Keys: ['location', 'name']
Name: B str
Sign: 0 int
Body: [ast.Literal(Location(begin=Position(filename='<string>', line=1, column=14), end=Position(filename='<string>', line=1, column=20)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=1, column=14), end=Position(filename='<string>', line=1, column=20)), 'dom', [ast.Variable(Location(begin=Position(filename='<string>', line=1, column=18), end=Position(filename='<string>', line

In [52]:
clingo.ast.parse_string("{ join(A,B) ; union(A,B)} :- dom(A), dom(B).", report)

#program base.
Type: ASTType.Program
Keys: ['location', 'name', 'parameters']
Name: base str
{ join(A,B); union(A,B) } :- dom(A); dom(B).
Type: ASTType.Rule
Keys: ['location', 'head', 'body']
Head: { join(A,B); union(A,B) }
Type: ASTType.Aggregate
Keys: ['location', 'left_guard', 'elements', 'right_guard']
Body: [ast.Literal(Location(begin=Position(filename='<string>', line=1, column=30), end=Position(filename='<string>', line=1, column=36)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=1, column=30), end=Position(filename='<string>', line=1, column=36)), 'dom', [ast.Variable(Location(begin=Position(filename='<string>', line=1, column=34), end=Position(filename='<string>', line=1, column=35)), 'A')], 0))), ast.Literal(Location(begin=Position(filename='<string>', line=1, column=38), end=Position(filename='<string>', line=1, column=44)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=1, column=38), end=Position(fil

In [53]:
clingo.ast.parse_string(" join(A,B) | union(A,B) :- dom(A), dom(B).", report)

#program base.
Type: ASTType.Program
Keys: ['location', 'name', 'parameters']
Name: base str
join(A,B); union(A,B) :- dom(A); dom(B).
Type: ASTType.Rule
Keys: ['location', 'head', 'body']
Head: join(A,B); union(A,B)
Type: ASTType.Disjunction
Keys: ['location', 'elements']
Body: [ast.Literal(Location(begin=Position(filename='<string>', line=1, column=28), end=Position(filename='<string>', line=1, column=34)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=1, column=28), end=Position(filename='<string>', line=1, column=34)), 'dom', [ast.Variable(Location(begin=Position(filename='<string>', line=1, column=32), end=Position(filename='<string>', line=1, column=33)), 'A')], 0))), ast.Literal(Location(begin=Position(filename='<string>', line=1, column=36), end=Position(filename='<string>', line=1, column=42)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=1, column=36), end=Position(filename='<string>', line=1, column=42

In [54]:
clingo.ast.parse_string("union(A,B) :- dom(A), dom(B), A=B.", report)

#program base.
Type: ASTType.Program
Keys: ['location', 'name', 'parameters']
Name: base str
union(A,B) :- dom(A); dom(B); A = B.
Type: ASTType.Rule
Keys: ['location', 'head', 'body']
Head: union(A,B)
Type: ASTType.Literal
Keys: ['location', 'sign', 'atom']
Atom: union(A,B)
Type: ASTType.SymbolicAtom
Keys: ['symbol']
Symbol: union(A,B)
Type: ASTType.Function
Keys: ['location', 'name', 'arguments', 'external']
External: 0 int
Name: union str
Arguments:
A
Type: ASTType.Variable
Keys: ['location', 'name']
Name: A str
B
Type: ASTType.Variable
Keys: ['location', 'name']
Name: B str
Sign: 0 int
Body: [ast.Literal(Location(begin=Position(filename='<string>', line=1, column=15), end=Position(filename='<string>', line=1, column=21)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=1, column=15), end=Position(filename='<string>', line=1, column=21)), 'dom', [ast.Variable(Location(begin=Position(filename='<string>', line=1, column=19), end=Position(filename='<st

In [55]:
clingo.ast.parse_string(":- dom(A), dom(B), A=B.", report)

#program base.
Type: ASTType.Program
Keys: ['location', 'name', 'parameters']
Name: base str
#false :- dom(A); dom(B); A = B.
Type: ASTType.Rule
Keys: ['location', 'head', 'body']
Head: #false
Type: ASTType.Literal
Keys: ['location', 'sign', 'atom']
Atom: #false
Type: ASTType.BooleanConstant
Keys: ['value']
Sign: 0 int
Body: [ast.Literal(Location(begin=Position(filename='<string>', line=1, column=4), end=Position(filename='<string>', line=1, column=10)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=1, column=4), end=Position(filename='<string>', line=1, column=10)), 'dom', [ast.Variable(Location(begin=Position(filename='<string>', line=1, column=8), end=Position(filename='<string>', line=1, column=9)), 'A')], 0))), ast.Literal(Location(begin=Position(filename='<string>', line=1, column=12), end=Position(filename='<string>', line=1, column=18)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=1, column=12), end=Pos

In [56]:
clingo.ast.parse_string("diff(A,A) :- domA(A), not domB(A).", report)

#program base.
Type: ASTType.Program
Keys: ['location', 'name', 'parameters']
Name: base str
diff(A,A) :- domA(A); not domB(A).
Type: ASTType.Rule
Keys: ['location', 'head', 'body']
Head: diff(A,A)
Type: ASTType.Literal
Keys: ['location', 'sign', 'atom']
Atom: diff(A,A)
Type: ASTType.SymbolicAtom
Keys: ['symbol']
Symbol: diff(A,A)
Type: ASTType.Function
Keys: ['location', 'name', 'arguments', 'external']
External: 0 int
Name: diff str
Arguments:
A
Type: ASTType.Variable
Keys: ['location', 'name']
Name: A str
A
Type: ASTType.Variable
Keys: ['location', 'name']
Name: A str
Sign: 0 int
Body: [ast.Literal(Location(begin=Position(filename='<string>', line=1, column=14), end=Position(filename='<string>', line=1, column=21)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=1, column=14), end=Position(filename='<string>', line=1, column=21)), 'domA', [ast.Variable(Location(begin=Position(filename='<string>', line=1, column=19), end=Position(filename='<string>

In [57]:
clingo.ast.parse_string("diff(A,A) :- domA(A), -domB(A).", report)

#program base.
Type: ASTType.Program
Keys: ['location', 'name', 'parameters']
Name: base str
diff(A,A) :- domA(A); -domB(A).
Type: ASTType.Rule
Keys: ['location', 'head', 'body']
Head: diff(A,A)
Type: ASTType.Literal
Keys: ['location', 'sign', 'atom']
Atom: diff(A,A)
Type: ASTType.SymbolicAtom
Keys: ['symbol']
Symbol: diff(A,A)
Type: ASTType.Function
Keys: ['location', 'name', 'arguments', 'external']
External: 0 int
Name: diff str
Arguments:
A
Type: ASTType.Variable
Keys: ['location', 'name']
Name: A str
A
Type: ASTType.Variable
Keys: ['location', 'name']
Name: A str
Sign: 0 int
Body: [ast.Literal(Location(begin=Position(filename='<string>', line=1, column=14), end=Position(filename='<string>', line=1, column=21)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=1, column=14), end=Position(filename='<string>', line=1, column=21)), 'domA', [ast.Variable(Location(begin=Position(filename='<string>', line=1, column=19), end=Position(filename='<string>', 

In [58]:
clingo.ast.parse_string("diff(A,A) :- domA(A), not not domB(A).", report)

#program base.
Type: ASTType.Program
Keys: ['location', 'name', 'parameters']
Name: base str
diff(A,A) :- domA(A); not not domB(A).
Type: ASTType.Rule
Keys: ['location', 'head', 'body']
Head: diff(A,A)
Type: ASTType.Literal
Keys: ['location', 'sign', 'atom']
Atom: diff(A,A)
Type: ASTType.SymbolicAtom
Keys: ['symbol']
Symbol: diff(A,A)
Type: ASTType.Function
Keys: ['location', 'name', 'arguments', 'external']
External: 0 int
Name: diff str
Arguments:
A
Type: ASTType.Variable
Keys: ['location', 'name']
Name: A str
A
Type: ASTType.Variable
Keys: ['location', 'name']
Name: A str
Sign: 0 int
Body: [ast.Literal(Location(begin=Position(filename='<string>', line=1, column=14), end=Position(filename='<string>', line=1, column=21)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=1, column=14), end=Position(filename='<string>', line=1, column=21)), 'domA', [ast.Variable(Location(begin=Position(filename='<string>', line=1, column=19), end=Position(filename='<str

In [59]:
clingo.ast.parse_string("diff(A,A) :- domA(A), not not -domB(A).", report)

#program base.
Type: ASTType.Program
Keys: ['location', 'name', 'parameters']
Name: base str
diff(A,A) :- domA(A); not not -domB(A).
Type: ASTType.Rule
Keys: ['location', 'head', 'body']
Head: diff(A,A)
Type: ASTType.Literal
Keys: ['location', 'sign', 'atom']
Atom: diff(A,A)
Type: ASTType.SymbolicAtom
Keys: ['symbol']
Symbol: diff(A,A)
Type: ASTType.Function
Keys: ['location', 'name', 'arguments', 'external']
External: 0 int
Name: diff str
Arguments:
A
Type: ASTType.Variable
Keys: ['location', 'name']
Name: A str
A
Type: ASTType.Variable
Keys: ['location', 'name']
Name: A str
Sign: 0 int
Body: [ast.Literal(Location(begin=Position(filename='<string>', line=1, column=14), end=Position(filename='<string>', line=1, column=21)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=1, column=14), end=Position(filename='<string>', line=1, column=21)), 'domA', [ast.Variable(Location(begin=Position(filename='<string>', line=1, column=19), end=Position(filename='<st

In [60]:
clingo.ast.parse_string("diff(A,A) :- domA(A), dom(0).", report)

#program base.
Type: ASTType.Program
Keys: ['location', 'name', 'parameters']
Name: base str
diff(A,A) :- domA(A); dom(0).
Type: ASTType.Rule
Keys: ['location', 'head', 'body']
Head: diff(A,A)
Type: ASTType.Literal
Keys: ['location', 'sign', 'atom']
Atom: diff(A,A)
Type: ASTType.SymbolicAtom
Keys: ['symbol']
Symbol: diff(A,A)
Type: ASTType.Function
Keys: ['location', 'name', 'arguments', 'external']
External: 0 int
Name: diff str
Arguments:
A
Type: ASTType.Variable
Keys: ['location', 'name']
Name: A str
A
Type: ASTType.Variable
Keys: ['location', 'name']
Name: A str
Sign: 0 int
Body: [ast.Literal(Location(begin=Position(filename='<string>', line=1, column=14), end=Position(filename='<string>', line=1, column=21)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=1, column=14), end=Position(filename='<string>', line=1, column=21)), 'domA', [ast.Variable(Location(begin=Position(filename='<string>', line=1, column=19), end=Position(filename='<string>', li

In [61]:
clingo.ast.parse_string("diff(A,A) :- domA(A), dom(a).", report)

#program base.
Type: ASTType.Program
Keys: ['location', 'name', 'parameters']
Name: base str
diff(A,A) :- domA(A); dom(a).
Type: ASTType.Rule
Keys: ['location', 'head', 'body']
Head: diff(A,A)
Type: ASTType.Literal
Keys: ['location', 'sign', 'atom']
Atom: diff(A,A)
Type: ASTType.SymbolicAtom
Keys: ['symbol']
Symbol: diff(A,A)
Type: ASTType.Function
Keys: ['location', 'name', 'arguments', 'external']
External: 0 int
Name: diff str
Arguments:
A
Type: ASTType.Variable
Keys: ['location', 'name']
Name: A str
A
Type: ASTType.Variable
Keys: ['location', 'name']
Name: A str
Sign: 0 int
Body: [ast.Literal(Location(begin=Position(filename='<string>', line=1, column=14), end=Position(filename='<string>', line=1, column=21)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=1, column=14), end=Position(filename='<string>', line=1, column=21)), 'domA', [ast.Variable(Location(begin=Position(filename='<string>', line=1, column=19), end=Position(filename='<string>', li

In [62]:
clingo.ast.parse_string("diff(A,A) :- domA(A), dom(-a).", report)

#program base.
Type: ASTType.Program
Keys: ['location', 'name', 'parameters']
Name: base str
diff(A,A) :- domA(A); dom(-a).
Type: ASTType.Rule
Keys: ['location', 'head', 'body']
Head: diff(A,A)
Type: ASTType.Literal
Keys: ['location', 'sign', 'atom']
Atom: diff(A,A)
Type: ASTType.SymbolicAtom
Keys: ['symbol']
Symbol: diff(A,A)
Type: ASTType.Function
Keys: ['location', 'name', 'arguments', 'external']
External: 0 int
Name: diff str
Arguments:
A
Type: ASTType.Variable
Keys: ['location', 'name']
Name: A str
A
Type: ASTType.Variable
Keys: ['location', 'name']
Name: A str
Sign: 0 int
Body: [ast.Literal(Location(begin=Position(filename='<string>', line=1, column=14), end=Position(filename='<string>', line=1, column=21)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=1, column=14), end=Position(filename='<string>', line=1, column=21)), 'domA', [ast.Variable(Location(begin=Position(filename='<string>', line=1, column=19), end=Position(filename='<string>', l

In [63]:
clingo.ast.parse_string("diff(0,1).", report)

#program base.
Type: ASTType.Program
Keys: ['location', 'name', 'parameters']
Name: base str
diff(0,1).
Type: ASTType.Rule
Keys: ['location', 'head', 'body']
Head: diff(0,1)
Type: ASTType.Literal
Keys: ['location', 'sign', 'atom']
Atom: diff(0,1)
Type: ASTType.SymbolicAtom
Keys: ['symbol']
Symbol: diff(0,1)
Type: ASTType.Function
Keys: ['location', 'name', 'arguments', 'external']
External: 0 int
Name: diff str
Arguments:
0
Type: ASTType.SymbolicTerm
Keys: ['location', 'symbol']
Symbol: 0
Type: Symbol
1
Type: ASTType.SymbolicTerm
Keys: ['location', 'symbol']
Symbol: 1
Type: Symbol
Sign: 0 int
Body: []
Type: ASTSequence


In [64]:
clingo.ast.parse_string("dom(A) :- dom(A).", report)

#program base.
Type: ASTType.Program
Keys: ['location', 'name', 'parameters']
Name: base str
dom(A) :- dom(A).
Type: ASTType.Rule
Keys: ['location', 'head', 'body']
Head: dom(A)
Type: ASTType.Literal
Keys: ['location', 'sign', 'atom']
Atom: dom(A)
Type: ASTType.SymbolicAtom
Keys: ['symbol']
Symbol: dom(A)
Type: ASTType.Function
Keys: ['location', 'name', 'arguments', 'external']
External: 0 int
Name: dom str
Arguments:
A
Type: ASTType.Variable
Keys: ['location', 'name']
Name: A str
Sign: 0 int
Body: [ast.Literal(Location(begin=Position(filename='<string>', line=1, column=11), end=Position(filename='<string>', line=1, column=17)), 0, ast.SymbolicAtom(ast.Function(Location(begin=Position(filename='<string>', line=1, column=11), end=Position(filename='<string>', line=1, column=17)), 'dom', [ast.Variable(Location(begin=Position(filename='<string>', line=1, column=15), end=Position(filename='<string>', line=1, column=16)), 'A')], 0)))]
Type: ASTSequence


In [65]:
clingo.ast.parse_string('rule(head(literal(sign(0), atom(symbol(function(name("domA"), arguments(term(symbol(0))), external(0)))))), body(elements())).', report)

#program base.
Type: ASTType.Program
Keys: ['location', 'name', 'parameters']
Name: base str
rule(head(literal(sign(0),atom(symbol(function(name("domA"),arguments(term(symbol(0))),external(0)))))),body(elements)).
Type: ASTType.Rule
Keys: ['location', 'head', 'body']
Head: rule(head(literal(sign(0),atom(symbol(function(name("domA"),arguments(term(symbol(0))),external(0)))))),body(elements))
Type: ASTType.Literal
Keys: ['location', 'sign', 'atom']
Atom: rule(head(literal(sign(0),atom(symbol(function(name("domA"),arguments(term(symbol(0))),external(0)))))),body(elements))
Type: ASTType.SymbolicAtom
Keys: ['symbol']
Symbol: rule(head(literal(sign(0),atom(symbol(function(name("domA"),arguments(term(symbol(0))),external(0)))))),body(elements))
Type: ASTType.Function
Keys: ['location', 'name', 'arguments', 'external']
External: 0 int
Name: rule str
Arguments:
head(literal(sign(0),atom(symbol(function(name("domA"),arguments(term(symbol(0))),external(0))))))
Type: ASTType.Function
Keys: ['loca